In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle

In [5]:
# 셀에서 값을 모두 출력
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import Image

In [6]:
## 설정
# 지수표기법 대신 숫자표기법
pd.options.display.float_format = '{:.2f}'.format

# 스타일
plt.style.use('seaborn-v0_8-darkgrid')

# 폰트
from matplotlib import font_manager
# - 맑은고딕 보통과 굵은 글꼴 경로
font_path_regular = "C:/Windows/Fonts/malgun.ttf"  # 맑은고딕 보통
font_path_bold = "C:/Windows/Fonts/malgunbd.ttf"  # 맑은고딕 굵게

# - 폰트 추가
font_manager.fontManager.addfont(font_path_regular)
font_manager.fontManager.addfont(font_path_bold)

# - 기본 폰트 설정 (맑은고딕 보통을 기본 폰트로 설정)
plt.rc('font', family='Malgun Gothic')  # 기본 폰트는 보통

# - 음수 부호 표시 설정
plt.rcParams['axes.unicode_minus'] = False  # 음수 부호 제대로 표시

# 데이터 프레임의 모든 레코드 출력
def print_all(obj, n=None):
    pd.set_option('display.max_row', n)
    display(obj)
    pd.set_option('display.max_row', 10)

# 모든 칼럼 출력
pd.set_option('display.max_columns', None)

In [7]:
# 분석 진행중인 임시 데이터셋 로드
data_filtered2 = pd.read_csv("tmp/clustered_data.csv")

In [9]:
# 분석 진행중인 임시 데이터셋 로드
data_filtered3 = pd.read_csv("tmp/clustered_data2.csv")

# 데이터 로드 및 탐색

In [5]:
# 데이터 로드: 2024년 9월 기준 서울 상가(상권) 정보
data = pd.read_csv('소상공인시장진흥공단_상가(상권)정보_서울_202409.csv')

C:\Users\sangw\AppData\Local\Temp\ipykernel_17832\1293301336.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('소상공인시장진흥공단_상가(상권)정보_서울_202409.csv')


In [6]:
# 분석 데이터세트의 규모를 줄이기 위해 분석 대상 지역을 강남구와 서초구로 한정
data_filtered = data[data['시군구명'].isin(['강남구', '서초구'])].reset_index(drop=True)

In [6]:
# 결측치 확인 (분석에 필요한 변수)
print(data_filtered[['상호명', '상권업종중분류명', '상권업종소분류명', '위도', '경도']].isna().sum())

상호명         0
상권업종중분류명    0
상권업종소분류명    0
위도          0
경도          0
dtype: int64


In [7]:
# 종합 탐색
def summary_df(df):
    summary = pd.DataFrame({
    '변수명':df.columns,
    '변수타입':df.dtypes.values,
    '유니크값 수':df.nunique().values,
    '결측치 개수':df.isna().sum().values
    })
    return summary

print_all(summary_df(data_filtered[['상호명', '상권업종중분류명', '상권업종소분류명', '위도', '경도']]))

,변수명,변수타입,유니크값 수,결측치 개수
0,상호명,object,82985,0
1,상권업종중분류명,object,75,0
2,상권업종소분류명,object,245,0
3,위도,float64,18915,0
4,경도,float64,18915,0


In [43]:
# 업종별 Count 찍어보기
cnt_by_industry = data_filtered['상권업종소분류명'].value_counts().reset_index()
print_all(cnt_by_industry.head(30))

,상권업종소분류명,count
0,경영 컨설팅업,7562
1,부동산 중개/대리업,5453
2,광고 대행업,4411
3,백반/한정식,3339
4,변호사,3094
5,카페,3059
6,건축 설계 및 관련 서비스업,2188
7,미용실,2134
8,세무사,2102
9,사진촬영업,1696


#  상권 클러스터링

In [7]:
# 위경도 좌표 배열로 변환
coord_loc = data_filtered[['위도', '경도']].to_numpy()

# DBSCAN 수행
dbsc = DBSCAN(eps=0.000005, min_samples=5, metric='haversine').fit(np.radians(coord_loc))

# cluster 라벨 추가
data_filtered['cluster'] = dbsc.labels_

"""# 결과를 pickle로 저장
import pickle
with open('tmp/dbsc_result.pkl', 'wb') as f:
    pickle.dump(dbsc, f)"""


"# 결과를 pickle로 저장\nimport pickle\nwith open('tmp/dbsc_result.pkl', 'wb') as f:\n    pickle.dump(dbsc, f)"

In [7]:
# DBSCAN 결과 불러오기
import pickle
with open('tmp/dbsc_result.pkl', 'rb') as f:
    dbsc = pickle.load(f)

In [177]:
# 상권별 규모 확인
stores_scale = pd.Series(dbsc.labels_).value_counts().reset_index().rename(columns={'index':'상권 번호', 'count':'규모'})
stores_scale[stores_scale['상권 번호'] != -1].head(20).to_clipboard(index=False)
stores_scale[stores_scale['상권 번호'] != -1].head(20)

,상권 번호,규모
0,38,3059
2,2,2155
3,83,1856
4,10,1834
5,27,1830
6,35,1392
7,109,1053
8,16,992
9,106,987
10,34,938


In [172]:
data_filtered2

,상호명,상권업종대분류명,상권업종중분류명,상권업종소분류명,위도,경도,cluster,cluster_analysis
0,무지개삼성약국,소매,의약·화장품 소매,약국,37.49,127.08,0,nan
1,나루공인중개사사무소,부동산,부동산 서비스,부동산 중개/대리업,37.50,127.04,1,nan
2,김병조법률사무소,과학·기술,법무관련,변호사,37.49,127.01,2,서초역 일대
3,스튜디온디,과학·기술,사진 촬영,사진촬영업,37.52,127.03,3,nan
4,송금당,소매,시계·귀금속 소매,시계/귀금속 소매업,37.47,127.05,4,양재 시민의 숲 역
...,...,...,...,...,...,...,...,...
89766,다정약국,소매,의약·화장품 소매,약국,37.53,127.03,66,nan
89767,우연,수리·개인,기타 개인,결혼 상담 서비스업,37.49,127.03,570,nan
89768,플라워테라피,소매,식물 소매,꽃집,37.49,127.08,369,nan
89769,이엔에이에스테이트컴퍼니,과학·기술,본사·경영 컨설팅,경영 컨설팅업,37.49,127.01,2,서초역 일대


In [168]:
# 클러스터별 주요 업종 분석
# 0) 분석 전 전처리
# - 편의상 분석에 필요한 변수만 남기기
data_filtered2 = data_filtered[['상호명', '상권업종대분류명' ,'상권업종중분류명' ,'상권업종소분류명' ,'위도', '경도', 'cluster']]

# - 각종 주점 종류는 '주점'으로 퉁치기
data_filtered2['상권업종소분류명'] = np.where(data_filtered2['상권업종중분류명'] == '주점', '주점', data_filtered2['상권업종소분류명'])

# - 각종 교육 시설은 '교육육'으로 퉁치기
data_filtered2['상권업종소분류명'] = np.where(data_filtered2['상권업종대분류명'] == '교육', '교육', data_filtered2['상권업종소분류명'])

# 1) 주요 비교 분석 대상 상권
data_filtered2['cluster_analysis'] = np.nan
data_filtered2['cluster_analysis'] = np.where(data_filtered2['cluster'] == 106, '강남역 11번 출구 일대', data_filtered2['cluster_analysis'])
data_filtered2['cluster_analysis'] = np.where(data_filtered2['cluster'] == 83, '논현_신논현', data_filtered2['cluster_analysis'])
data_filtered2['cluster_analysis'] = np.where(data_filtered2['cluster'] == 38, '가로수길_압구정 일대', data_filtered2['cluster_analysis'])
data_filtered2['cluster_analysis'] = np.where(data_filtered2['cluster'] == 161, '대치동 학원가', data_filtered2['cluster_analysis'])
data_filtered2['cluster_analysis'] = np.where(data_filtered2['cluster'] == 2, '서초역 일대', data_filtered2['cluster_analysis'])
data_filtered2['cluster_analysis'] = np.where(data_filtered2['cluster'] == 10, '압구정 로데오 일대', data_filtered2['cluster_analysis'])
data_filtered2['cluster_analysis'] = np.where(data_filtered2['cluster'] == 93, '서래마을 카페거리 일대', data_filtered2['cluster_analysis'])
data_filtered2['cluster_analysis'] = np.where(data_filtered2['cluster'] == 4, '양재 시민의 숲 역', data_filtered2['cluster_analysis'])

C:\Users\sangw\AppData\Local\Temp\ipykernel_17832\1666455481.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered2['상권업종소분류명'] = np.where(data_filtered2['상권업종중분류명'] == '주점', '주점', data_filtered2['상권업종소분류명'])
C:\Users\sangw\AppData\Local\Temp\ipykernel_17832\1666455481.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered2['상권업종소분류명'] = np.where(data_filtered2['상권업종대분류명'] == '교육', '교육', data_filtered2['상권업종소분류명'])
C:\Users\sangw\AppData\Local\Temp\ipykernel_17832\1666455481.py:13: S

In [170]:
# CSV 저장
data_filtered2.to_csv("tmp/clustered_data.csv", index=False)

# 유사 상권 그룹화

### 상권별 업종의 비율 계산

In [8]:
# 상권에 포함되지 않은 노이즈 점포는 분석 대상에서 제외
data_filtered3 = data_filtered2[data_filtered2['cluster'] != -1]

# 업종 앞에 대, 중, 소 구분할 수 있게 표시
data_filtered3['상권업종대분류명_2'] = '대_' + data_filtered3['상권업종대분류명']
data_filtered3['상권업종중분류명_2'] = '중_' + data_filtered3['상권업종중분류명']
data_filtered3['상권업종소분류명_2'] = '소_' + data_filtered3['상권업종소분류명']

# 대분류, 중분류, 소분류를 모두 고려한 업종 비율 계산
# - 대분류
L_upjong_counts = data_filtered3.groupby(['cluster', '상권업종대분류명_2']).size().unstack(fill_value=0)
L_upjong_ratio = L_upjong_counts.div(L_upjong_counts.sum(axis=1), axis=0)

# - 중분류
M_upjong_counts = data_filtered3.groupby(['cluster', '상권업종중분류명_2']).size().unstack(fill_value=0)
M_upjong_ratio = M_upjong_counts.div(M_upjong_counts.sum(axis=1), axis=0)

# - 소분류
S_upjong_counts = data_filtered3.groupby(['cluster', '상권업종소분류명_2']).size().unstack(fill_value=0)
S_upjong_ratio = S_upjong_counts.div(S_upjong_counts.sum(axis=1), axis=0)

# 대분류, 중분류, 소분류 비율 데이터프레임 결합
combined_upjong_ratio = pd.concat([L_upjong_ratio, M_upjong_ratio, S_upjong_ratio], axis=1)

C:\Users\sangw\AppData\Local\Temp\ipykernel_26180\2218078363.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered3['상권업종대분류명_2'] = '대_' + data_filtered3['상권업종대분류명']
C:\Users\sangw\AppData\Local\Temp\ipykernel_26180\2218078363.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered3['상권업종중분류명_2'] = '중_' + data_filtered3['상권업종중분류명']
C:\Users\sangw\AppData\Local\Temp\ipykernel_26180\2218078363.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

### 코사인 유사도 측정

In [184]:
# 업종 비율 데이터를 벡터화
category_vectors = combined_upjong_ratio.to_numpy()

# 코사인 유사도 계산
cosine_matrix = cosine_similarity(category_vectors)

# 결과를 데이터프레임으로 변환
cosine_df = pd.DataFrame(cosine_matrix, index=combined_upjong_ratio.index, columns=combined_upjong_ratio.index)

In [185]:
cosine_df

cluster  0     1     2     3     4     5     6     7     8     9     10    \
cluster                                                                     
0        1.00  0.42  0.47  0.72  0.81  0.80  0.39  0.72  0.81  0.53  0.92   
1        0.42  1.00  0.75  0.73  0.69  0.17  0.77  0.38  0.09  0.10  0.37   
2        0.47  0.75  1.00  0.72  0.70  0.31  0.49  0.46  0.24  0.18  0.46   
3        0.72  0.73  0.72  1.00  0.93  0.51  0.57  0.63  0.47  0.36  0.76   
4        0.81  0.69  0.70  0.93  1.00  0.59  0.52  0.66  0.58  0.41  0.80   
...       ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
1618     0.55  0.41  0.55  0.63  0.59  0.38  0.22  0.46  0.40  0.32  0.62   
1619     0.03  0.01  0.01  0.03  0.03  0.01  0.02  0.07  0.03  0.05  0.03   
1620     0.32  0.66  0.58  0.73  0.59  0.13  0.43  0.32  0.07  0.21  0.38   
1621     0.33  0.75  0.65  0.75  0.70  0.13  0.47  0.34  0.08  0.19  0.37   
1622     0.34  0.20  0.23  0.42  0.38  0.12  0.11  0.13  0.25  0.52  0.43   

cluster  11    12    13    14    15    16    17    18    19    20    21    \
cluster                                                                     
0        0.64  0.65  0.79  0.88  0.40  0.66  0.88  0.40  0.91  0.42  0.84   
1        0.52  0.20  0.69  0.59  0.92  0.85  0.55  0.75  0.60  0.88  0.33   
2        0.51  0.25  0.73  0.61  0.78  0.77  0.60  0.98  0.68  0.71  0.31   
3        0.61  0.42  0.92  0.84  0.80  0.83  0.87  0.67  0.87  0.81  0.50   
4        0.68  0.48  0.96  0.86  0.73  0.82  0.89  0.64  0.90  0.70  0.62   
...       ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
1618     0.41  0.37  0.58  0.61  0.48  0.53  0.64  0.50  0.64  0.48  0.34   
1619     0.05  0.05  0.02  0.02  0.01  0.02  0.03  0.00  0.07  0.01  0.02   
1620     0.39  0.16  0.61  0.48  0.78  0.62  0.54  0.56  0.49  0.83  0.17   
1621     0.42  0.16  0.69  0.49  0.86  0.67  0.54  0.62  0.52  0.84  0.20   
1622     0.44  0.20  0.43  0.28  0.34  0.27  0.47  0.21  0.31  0.33  0.31   

cluster  22    23    24    25    26    27    28    29    30    31    32    \
cluster                                                                     
0        0.56  0.78  0.83  0.59  0.85  0.43  0.46  0.70  0.52  0.82  0.49   
1        0.59  0.60  0.50  0.35  0.33  0.71  0.92  0.85  0.87  0.80  0.57   
2        0.49  0.46  0.74  0.50  0.40  0.99  0.73  0.75  0.89  0.73  0.50   
3        0.70  0.65  0.70  0.49  0.65  0.67  0.79  0.87  0.82  0.88  0.56   
4        0.65  0.68  0.77  0.52  0.69  0.64  0.69  0.87  0.79  0.89  0.58   
...       ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
1618     0.38  0.49  0.56  0.33  0.62  0.52  0.46  0.53  0.53  0.57  0.39   
1619     0.02  0.02  0.01  0.02  0.08  0.02  0.02  0.02  0.01  0.03  0.01   
1620     0.54  0.39  0.33  0.33  0.31  0.54  0.80  0.65  0.68  0.63  0.37   
1621     0.54  0.38  0.35  0.34  0.30  0.60  0.82  0.71  0.77  0.66  0.45   
1622     0.41  0.26  0.20  0.43  0.32  0.23  0.31  0.32  0.22  0.33  0.24   

cluster  33    34    35    36    37    38    39    40    41    42    43    \
cluster                                                                     
0        0.76  0.52  0.86  0.64  0.74  0.85  0.88  0.63  0.52  0.76  0.71   
1        0.77  0.88  0.65  0.72  0.86  0.63  0.73  0.84  0.86  0.85  0.64   
2        0.75  0.75  0.61  0.58  0.78  0.65  0.65  0.73  0.66  0.78  0.54   
3        0.97  0.88  0.79  0.73  0.90  0.93  0.85  0.93  0.79  0.90  0.72   
4        0.93  0.78  0.83  0.71  0.88  0.91  0.88  0.85  0.72  0.92  0.75   
...       ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
1618     0.62  0.52  0.52  0.45  0.60  0.69  0.60  0.57  0.44  0.58  0.37   
1619     0.04  0.02  0.04  0.02  0.04  0.03  0.06  0.02  0.02  0.04  0.02   
1620     0.70  0.82  0.49  0.59  0.70  0.63  0.54  0.78  0.73  0.64  0.43   
1621     0.74  0.84  0.50  0.61  0.72  0.63  0.56  0.80  0.75  0.70  0.49   
1622     0.42  0.33  0.32  0.38  0.32  0.47  0.36  0.39  0.27  0.32  0

### 상권별 군집화

In [239]:
from sklearn.cluster import AgglomerativeClustering

# 1. 코사인 유사도 행렬을 거리 행렬(1 - 코사인 유사도)로 변환
distance_matrix = 1 - cosine_matrix

# 2. 계층적 군집화 수행
cluster_model = AgglomerativeClustering(n_clusters=None, metric='precomputed', linkage='average', distance_threshold=0.3)  # 1 - 0.7 = 0.3
cluster_labels = cluster_model.fit_predict(distance_matrix)

# 3. 데이터프레임에 클러스터 레이블 추가
combined_upjong_ratio['cluster_group'] = cluster_labels

# 4. 클러스터별 평균 유사도 계산
cluster_avg_similarity = combined_upjong_ratio.groupby('cluster_group').apply(lambda x: cosine_df.loc[x.index, x.index].mean().mean())


In [240]:
# 상권 그룹별 유사도 기초통계량
cluster_avg_similarity.describe()

count   351.00
mean      0.93
std       0.08
min       0.76
25%       0.85
50%       1.00
75%       1.00
max       1.00
dtype: float64

In [247]:
# 각 점포마다 어떤 상권 그룹(cluster_group)에 속하는지 확인하는 코드

# 전체 상권 그룹의 고유값을 추출
cluster_group_vals = combined_upjong_ratio['cluster_group'].unique()

# data_filtered3에 'cluster_group' 열을 추가하고, 초기값을 NaN으로 설정
data_filtered3['cluster_group'] = np.nan

# 각 상권 그룹(cluster_group)에 대해 반복문 실행
for cluster_group in cluster_group_vals:
    # 특정 상권 그룹에 포함되는 개별 상권(cluster)들을 추출
    clusters_in_group = combined_upjong_ratio[combined_upjong_ratio['cluster_group'] == cluster_group].index

    # data_filtered3의 'cluster' 값이 clusters_in_group에 포함되는 경우,
    # 해당하는 'cluster_group' 값을 부여
    data_filtered3['cluster_group'] = np.where(
        data_filtered3['cluster'].isin(clusters_in_group), 
        cluster_group, 
        data_filtered3['cluster_group']
    )

# 정수로 변환
data_filtered3['cluster_group'] = data_filtered3['cluster_group'].astype(int)

C:\Users\sangw\AppData\Local\Temp\ipykernel_17832\623763284.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered3['cluster_group'] = np.nan
C:\Users\sangw\AppData\Local\Temp\ipykernel_17832\623763284.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered3['cluster_group'] = np.where(
C:\Users\sangw\AppData\Local\Temp\ipykernel_17832\623763284.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

In [267]:
# 데이터 저장
data_filtered3.to_csv('tmp/clustered_data2.csv', index=False)

# 최적 입지 선정 분석

In [47]:
# 분석 대상 상권 그룹: 34, 4, 42, 32, 158, 66
analysis_target = [34, 4, 42, 32, 158, 66]

# 분석 대상 상권 그룹의 데이터만 추출
data_filtered4 = data_filtered3[data_filtered3['cluster_group'].isin(analysis_target)]

In [55]:
# 상권 그룹 별 성형외과 관련 업종 분포 탐색
# 1. 관련 업종 리스트 정의
plastic_related = ['성형외과 의원', '화장품 소매업', '피부 관리실', '약국', '피부/비뇨기과 의원']

# 2. 그룹별 전체 점포 수 계산
num_stores_per_group = data_filtered4.groupby('cluster_group').size()

# 3. 관련 업종이 포함된 점포 수 계산
num_related_stores_per_group = data_filtered4[data_filtered4['상권업종소분류명'].isin(plastic_related)].groupby('cluster_group').size()

# 4. 성형외과 개수 확인
num_plastic_per_group = data_filtered4[data_filtered4['상권업종소분류명'] == '성형외과 의원'].groupby('cluster_group').size()

# 5. 상권 개수
num_cluster = data_filtered4.groupby(['cluster_group'])['cluster'].nunique()


plastic_summary = pd.DataFrame({
    '상권 개수': num_cluster,
    '전체 점포수':num_stores_per_group,
    '성형외과 관련업종 점포수': num_related_stores_per_group,
    '성형외과 점포 수': num_plastic_per_group
})

plastic_summary.to_clipboard()


In [87]:
# 상권별 경쟁도 계산
# 1. 상권별 점포 수
num_stores_per_cluster = data_filtered4.groupby(['cluster_group', 'cluster']).size()

# 2. 상권별 관련 업종 점포 수
num_related_per_cluster = data_filtered4[data_filtered4['상권업종소분류명'].isin(plastic_related)].groupby(['cluster_group', 'cluster']).size()

# 3. 상권별 성형외과 점포 수
num_plastic_per_cluster = data_filtered4[data_filtered4['상권업종소분류명'] == '성형외과 의원'].groupby(['cluster_group', 'cluster']).size()

# 4. 상권별 관련 업종 비율
ratio_related_per_cluster = (num_related_per_cluster / num_stores_per_cluster).fillna(0)

# 5. 상권별 성형외과 비율
ratio_plastic_per_cluster = (num_plastic_per_cluster / num_stores_per_cluster).fillna(0)

# 6. 경쟁도 계산
competition_ratio = (ratio_plastic_per_cluster/ratio_related_per_cluster).fillna(0)

# 7. 경쟁도 요약 테이블 
competition_summary = pd.DataFrame({
    '상권 그룹': num_stores_per_cluster.index.get_level_values('cluster_group'),
    '상권': num_stores_per_cluster.index.get_level_values('cluster'),
    '전체 점포 수': num_stores_per_cluster.values,
    '성형외과 관련업종 점포 수': num_related_per_cluster.reindex(num_stores_per_cluster.index, fill_value=0).values,
    '성형외과 점포 수': num_plastic_per_cluster.reindex(num_stores_per_cluster.index, fill_value=0).values,
    '경쟁도': competition_ratio.values
}).sort_values(['상권 그룹', '경쟁도'], ascending=[True, False])

# 8. 성형외과 관련업종 점포 수가 20개 이상인 상권만 필터링
competition_summary_filtered = competition_summary[competition_summary['성형외과 관련업종 점포 수'] > 20]
competition_summary_filtered.to_clipboard(index=False)